In [47]:
import pandas as pd
Tankdcit = dict() # overall Tank

def Calculation_by_Input_Time(TankList):
    def timeperiod():
        from datetime import date
        x = input("Enter Time Period (yyyy.mm.dd): ")
        y = x.split(".")


        year = int(y[0])
        month = int(y[1])
        day = int(y[2])

        d0 = date(1900, 1, 1)
        d1 = date(year, month, day)
        delta = d1 - d0
        return(delta.days+2)

    startTime = timeperiod()
    endTime = timeperiod()

    def runtime_cal(Data, startTime, endTime, levels = False):


            DetailDict = dict() # attribute of equipment ex: running time
            timeList = list() # Time column in Data
            statusList = list() # status column in Data
            status = 0 # status
            recodTime = 0 # recording temp time
            runningTime = 0
            idelTIme = 0
            normal = 1

            for i in range(len(Data)):
                if Data.iat[i, 0] != ' ' or Data.iat[i, 1] != ' ':
                    try:

                        if float(Data.iat[i, 0]) >= startTime and float(Data.iat[i, 0]) <= endTime:
                            timeList.append(Data.iat[i, 0])
                            statusList.append(Data.iat[i,1])
                    except ValueError: 
                        timeList.append(Data.iat[i, 0])
                        statusList.append(Data.iat[i,1])


            if type(timeList[0]) == str:
                try:
                    if type(float(timeList[0])) == float:
                        normal = 1
                except ValueError:
                    normal = 0
                    DetailDict["idelTime"] = timeList[0]
                    DetailDict["totalTime"] = timeList[0]
                    DetailDict["runningTime"] = timeList[0]
                    DetailDict["Utilization"] = timeList[0]

            if normal == 1:
                if levels == False:

                    if statusList[0] == "RUNNING" or  statusList[0] == "1":
                        status = 1
                        recodTime = float(timeList[0])
                    else:
                        status = 0
                    for j in range(len(statusList)):
                        if statusList[j] == "RUNNING" or statusList[j] == "1":
                            if status == 0:
                                recodTime = float(timeList[j])
                                status = 1
                            else:
                                pass
                        elif statusList[j] == "STOPPED" or statusList[j] == "0":
                            if status == 0:
                                pass
                            else:
                                runningTime = runningTime + (float(timeList[j]) - recodTime)*24
                                status = 0

                    if statusList[len(statusList)-1] == "RUNNING" or statusList[len(statusList)-1] == "1":
                        runningTime = (float(timeList[len(statusList)-1]) - recodTime)*24 + runningTime
                elif levels == True:

                    try:

                        if float(statusList[0]) > 50.00:
                            status = 1
                            recodTime = float(timeList[0])
                        else:
                            status = 0

                        for j in range(len(statusList)):

                            try:                        
                                if float(statusList[j]) > 50.00:
                                    if status == 0:
                                        recodTime = float(timeList[j])
                                        status = 1
                                        #print(">")
                                    else:
                                        pass
                                elif float(statusList[j]) < 50.00:
                                    if status == 0:
                                        pass
                                    else:
                                        runningTime = runningTime + (float(timeList[j]) - recodTime)*24
                                        status = 0
                            except ValueError:
                                pass
                        if float(statusList[len(statusList)-1]) > 50.00:
                            runningTime =  (float(timeList[len(statusList)-1]) - recodTime)*24 + runningTime
                    except ValueError:
                        pass

                idelTime = (float(timeList[len(timeList)-1])  - float(timeList[0]))*24 - runningTime
                totalTime = (float(timeList[len(timeList)-1]) - float(timeList[0]))*24


                DetailDict["idelTime"] = idelTime
                DetailDict["totalTime"] = totalTime
                DetailDict["runningTime"] = runningTime
                DetailDict["Utilization"] = runningTime/totalTime


            return DetailDict

    def TransformTimeFormat(Data):
        from datetime import date

        timeList = list()
        for i in range(len(Data)):
            if Data.iat[i, 0] != ' ' or Data.iat[i, 1] != ' ':
                try:    
                    timeList.append(Data.iat[i, 0])



                except ValueError: 
                    timeList.append(Data.iat[i, 0])

        #print(timeList)
        d0 = date(1900, 1, 1)

        new_timeList = list()# handle Transfomred Time Format
        for k in range(len(timeList)):
            year = timeList[k].year
            month = timeList[k].month
            day = timeList[k].day
            hour = timeList[k].hour
            minute = timeList[k].minute
            second = timeList[k].second


            d1 = date(year, month, day)
            delta = d1 - d0
            new_timeList. append(delta.days+2+hour/24+minute/24/60+second/24/60/60)
            #print(delta.days+2+hour/24+minute/24/60+second/24/60/60)
        return new_timeList
    print(TankList)
    
    for i in range(len(TankList)):

        Filename  = "C:/Users/johnn/Python/Industry Practicum/new data/" + "Tank " + str(TankList[i]) + ".csv"
        data = pd.read_csv(Filename)

        agi = data[["AgiTime", "Agitator Status"]]
        level = data[["LevelTime", "Tank Level"]]
        jack = data[["JackPTime", "Tank Jacket Pump"]]
        samp = data[["SamPTime", "Tank Sample Pump"]]
        con = data[["ConPTime", "Condenser Circulation Pump"]]

        agi = agi.dropna()
        level = level.dropna()
        jack = jack.dropna()
        samp = samp.dropna()
        con = con.dropna()

        # Handle different time format

        try:
            if float(agi.iat[0,0]) > 4000:
                pass

        except:
            try:
                agi['AgiTime'] = pd.to_datetime(agi['AgiTime'])
                agi['AgiTime'] = pd.DataFrame(TransformTimeFormat(agi))
            except:
                pass
        try:
            if float(level.iat[0,0]) > 4000:
                pass
        except:
            try:
                level['LevelTime'] = pd.to_datetime(level['LevelTime'])
                level['LevelTime'] = pd.DataFrame(TransformTimeFormat(level))
            except:
                pass

        try:
            if float(jack.iat[0,0]) > 4000:
                pass
        except:
            try:
                jack['JackPTime'] = pd.to_datetime(jack['JackPTime'])
                jack['JackPTime'] = pd.DataFrame(TransformTimeFormat(jack))
            except:
                pass 

        try:    
            if float(samp.iat[0,0]) > 4000:
                pass
        except:
            try:
                samp['SamPTime'] = pd.to_datetime(samp['SamPTime'])
                samp['SamPTime'] = pd.DataFrame(TransformTimeFormat(samp))
            except:
                pass
        try:    
            if float(con.iat[0,0]) > 4000:
                pass
        except:
            try:
                con['ConPTime'] = pd.to_datetime(con['ConPTime'])
                con['ConPTime'] = pd.DataFrame(TransformTimeFormat(con))
            except:
                pass



        TankEquipDict = dict() # tank equipment Dict ex: agitator, level


        TankEquipDict["Agitator"] = runtime_cal(agi, startTime, endTime)
        TankEquipDict["TankLevel"] = runtime_cal(level, startTime, endTime, levels = True)
        TankEquipDict["JacketPump"] = runtime_cal(jack, startTime, endTime)
        TankEquipDict["SamplePump"] = runtime_cal(samp, startTime, endTime)
        TankEquipDict["CondenserPump"] = runtime_cal(con, startTime, endTime)


        Tankdcit[TankList[i]] = TankEquipDict
    Colname = ["Tank No", "Agitator Running Time", "Agitator Idle Time", "Agitator Utilization Rate", "Tank Level Running Time", "Tank Level Idle Time", "Tank Level Utilization Rate",
              "JacketPump Running Time", "JacketPump idle Time", "JacketPump Utilization Rate", "SamplePump Running Time", "SamplePump Idle Time", "SamplePump Utilization Rate",
              "CondenserPump Running Time", "CondenserPump Idle Time", "CondenserPump Utilization Rate"]
    report_file = pd.DataFrame(columns = Colname )
    for i in Tankdcit:
        tempList = list()
        tempList.append(i)
        tempList.append(Tankdcit[i]['Agitator']['runningTime'])
        tempList.append(Tankdcit[i]['Agitator']['idelTime'])
        tempList.append(Tankdcit[i]['Agitator']['Utilization'])
        tempList.append(Tankdcit[i]['TankLevel']['runningTime'])
        tempList.append(Tankdcit[i]['TankLevel']['idelTime'])
        tempList.append(Tankdcit[i]['TankLevel']['Utilization'])
        tempList.append(Tankdcit[i]['JacketPump']['runningTime'])
        tempList.append(Tankdcit[i]['JacketPump']['idelTime'])
        tempList.append(Tankdcit[i]['JacketPump']['Utilization'])
        tempList.append(Tankdcit[i]['SamplePump']['runningTime'])
        tempList.append(Tankdcit[i]['SamplePump']['idelTime'])
        tempList.append(Tankdcit[i]['SamplePump']['Utilization'])
        tempList.append(Tankdcit[i]['CondenserPump']['runningTime'])
        tempList.append(Tankdcit[i]['CondenserPump']['idelTime'])
        tempList.append(Tankdcit[i]['CondenserPump']['Utilization'])
        c = pd.Series(tempList, index = Colname)
        report_file = report_file.append(c, ignore_index=True)
    return report_file

def output_report(directory, file):
    import pandas as pd
    directory = directory + '/Utilzation Report by Time period.csv'
    file.to_csv(directory)